# ***Bayes Theory applied on medical use case***

In [ ]:

# If pgmpy is not installed, uncomment next line
# !pip install pgmpy
from BNReasoner import BNReasoner
from BayesNet import BayesNet
import networkx as nx
import pandas as pd

# Instantiation of the Medical use case
net = BayesNet()
bn = BNReasoner(net)
net.load_from_bifxml('medical_use_case.BIFXML')

# All nodes in the use case 
net.get_all_variables()


['Junk-Food',
 'Stress',
 'Job-loss',
 'Weight-gain',
 'Insulin-resistance',
 'Anxiety',
 'High-blood-pressure',
 'Alcohol-consumption',
 'Trauma',
 'Diabetes',
 'Depression',
 'Stroke']

#**Prior marginal**

---



In the examples below we demonstrate the calculation of prior marginals. Without any evidence, the following prior marginals can be calculated:



Probability of Job-loss




In [ ]:
bn.marginal_distribution(['Job-loss'],{})



,Job-loss,p
1,True,0.8
0,False,0.2


Probability of Alcohol-consumption

In [ ]:
bn.marginal_distribution(['Alcohol-consumption'] , {})

,Alcohol-consumption,p
1,True,0.84
0,False,0.16



Probability of Diabetes-consumption and Anxiety


In [ ]:
bn.marginal_distribution(['Diabetes', 'Anxiety'],{})

,Diabetes,Anxiety,p
3,True,True,0.605208
2,True,False,0.237363
1,False,True,0.113792
0,False,False,0.043637


#**Posterior marginal**

---



In these examples below we demonstrate the calculation of posterior marginals. Without any evidence, the following prior marginals can be calculated:

The probability of Alcohol consumption if it is known that Job-loss is true. It is clear that the probability of someone consuming alcohol is much higher (0.93) if we know that this person lost his job, than the probability if we don't know whether this person lost his job (0.84), as seen in the prior marginal.


In [ ]:
bn.marginal_distribution(['Alcohol-consumption'],{'Job-loss': True})


,Alcohol-consumption,p
1,True,0.93
0,False,0.07


The probability of trauma if we know that Alcohol-consumption is true, is 0.86. 

In [ ]:
bn.marginal_distribution(['Trauma'],{'Alcohol-consumption': True})


,Trauma,p
1,True,0.864286
0,False,0.135714


This probability is slightly decreasing if we know that another direct cause of Trauma, namely Stress, is not the case. Then the probability of having a Trauma is 0.7.

In [ ]:
bn.marginal_distribution(['Trauma'],{'Alcohol-consumption': True, 'Stress':False})


,Trauma,p
1,True,0.7
0,False,0.3


# **MAP**


---



In these examples we show what the most probable instantiation is of a variable in the BN given some evidence.

Looking at Weight-gain, we could take into account the evidence of Junk-food. If we know someone eats junk food, then the instantiantiation of weight-gain is most probably False (0.43 is the maximum value of all probable instantions of Weight-gain)

In [ ]:

bn.map_mpe_estimation(pd.Series({"Junk-Food": False}), ["Weight-gain"])

,Weight-gain,p
0,False,0.43


If this person eats junk food, the most probable instantiation of Weight-gain is most probably True (again, considering the instantiation with the highest probability given the evidence).

In [ ]:
bn.map_mpe_estimation(pd.Series({"Junk-Food": True}), ["Weight-gain"])

,Weight-gain,p
0,True,0.32


#**MPE**

In the following example, the functionality of the MPE function is showed. MPE is a special case of MAP where no evidence is given. So, the most probable instantiations of Weight-gain and Stress without any given evidence are as follows.

In [ ]:
bn.map_mpe_estimation(pd.Series([],dtype=pd.StringDtype()) , ["Weight-gain","Stress"])

,Weight-gain,Stress,p
0,False,False,0.37


This is indeed the most probable instantiation of both Weight-gain and Stress, since this combination (False, False) has the highest probability out of a prior marginal_distribution:

In [ ]:
bn.marginal_distribution(['Weight-gain', 'Stress'])


,Weight-gain,Stress,p
3,True,True,0.36
2,True,False,0.03
1,False,True,0.24
0,False,False,0.37


#**D-separation**


In the examples below, the functionality of d-separation is shown. Job loss is not independent of trauma if alcohol-consumption is known. That is, because there is an active path from Job-loss to Trauma through Stress.

In [ ]:
bn.d_separation_with_pruning(['Job-loss'],['Alcohol-consumption'],['Trauma'])



False

 But, if Stress is added to the evidence, job loss is independent of trauma. 

In [ ]:
bn.d_separation_with_pruning(['Job-loss'],['Alcohol-consumption', 'Stress'],['Trauma'])

True

A convergent valve ->W<- is only blocked if it is not in the evidence set and neither are its descendents. For example, Job-loss is not d-seperated from Stress by Alcohol-consumption:

In [ ]:
bn.d_separation_with_pruning(['Job-loss'],['Alcohol-consumption'],['Stress'])

False

But if Alcohol consumption or any if its descendents (Trauma) is not given, the path to Stress is blocked:

In [ ]:
bn.d_separation_with_pruning(['Job-loss'],[],['Stress'])

True

In [ ]:
bn.d_separation_with_pruning(['Job-loss'],['Trauma'],['Stress'])

False